# ADC Simulationn (Linearität)
Version: 12.2020

Mit dieser Simulation sollen grundsätzlich RunUp und RunDown Funktionen

# Library laden


In [1]:
import SimADC_V1 as adc
import pandas as pd
import plotly.graph_objects as px

import plotly.io as pio
pio.renderers.default = 'iframe'


## Simulation starten


## Diagramm erstellen
Vout am Integrator

Beipiele und Dokumention finden sie auf der folgenden Seite:
https://plotly.com/python/<br>
https://plotly.com/python/bar-charts/#basic-bar-chart-with-plotlygraphobjects

In [2]:
# Ausgabe 100 Werten

x_vals = []
y_vals= []
integrator = adc.Integrator()
integrator.Vin = 5.0

integrator.VrefN = -7.0448
integrator.VrefP =  7.0448
integrator.ChargingInjection = 100.0 # pC  # DG411

integrator.R_VrefN = 20.0  # [kOhm]
integrator.R_VrefP = 20.0  # [kOhm]
integrator.R_Vin =   40.0  # [kOhm]
integrator.C = 2.2 # in nF



cpu = adc.CPU()


# CPU Ticks = 0.2 us => 5 Mhz
cpu.RunUpCycles = 6250   # 
cpu.RunUpCnt    = 50    #  50 * 0.2 = 10us
cpu.WaveForm    = 0    # Zwischenkompensation

# Auflösung: 6250 * 30  = 1.875.000
# Bei Vin +/-10V = 20V / Aufslösung = 10 uV (theoretisch) 


cpu.Connect( integrator) # Kontrolle für den Integrator an die CPU übergeben
cpu.ResidueADC = adc.ResidueADC(cpu, 12, 3.0) # Reste ADC mit n bit erzeugen

# Kalibration Array 
# Parameter: Vadc Vin gemessen ADC
# Parameter: Vtrue Vin real

calibration_points = [
    (-6.064076321123799, -6.0),  # -Vref
    (-0.06420449762620192, 0.0),            # 0 V
    (5.935670086298077  ,   6.0)    # +Vref
]

# Initialisiere die Kalibrierung
#cpu.Calibration = adc.ADCCalibration(calibration_points)





integrator.Vin = -5.0
for a in range(0,200+1):
    
    cpu.Reset()
    
    #cpu.AutoZero()
    cpu.RunUp()
    cpu.RunDown()
    cpu.CalcVin() # Berechnet den Wert der CPU - Vin 
    

    
    delta = (integrator.Vin - cpu.Vin) 
    x_vals.append( integrator.Vin)
    y_vals.append( delta)
    
    print("delta  : ", delta , " V")
    
    integrator.Vin = integrator.Vin + 0.05
    
    
    
    
    

delta  :  -0.1363165215000004  V
delta  :  -0.13390510099999986  V
delta  :  -0.13625800850000047  V
delta  :  -0.13384658799999993  V
delta  :  -0.13143516750000028  V
delta  :  -0.12902374699999974  V
delta  :  -0.12661232650000098  V
delta  :  -0.1289652340000007  V
delta  :  -0.12655381350000106  V
delta  :  -0.1241423930000014  V
delta  :  -0.12173097250000176  V
delta  :  -0.12408594250000249  V
delta  :  -0.12167452200000195  V
delta  :  -0.11926310150000319  V
delta  :  -0.11685168100000176  V
delta  :  -0.11444026050000211  V
delta  :  -0.11679316800000183  V
delta  :  -0.11438174750000307  V
delta  :  -0.11197032700000253  V
delta  :  -0.10955890650000333  V
delta  :  -0.11191181400000305  V
delta  :  -0.10950039350000385  V
delta  :  -0.1070889730000033  V
delta  :  -0.10467755250000366  V
delta  :  -0.10226613200000445  V
delta  :  -0.10461903950000417  V
delta  :  -0.10220761900000497  V
delta  :  -0.09979619850000443  V
delta  :  -0.09738477800000478  V
delta  :  -0.09973

In [3]:

#print(x_vals)


# neues Diagramm erstellen mit Titel
fig = px.Figure()


# Linie anzeigen
fig.add_trace( px.Scatter( x=x_vals, y=y_vals , name="Vout" )  )

fig.update_layout(
   title_text='Abweichung der Messung',
    yaxis_title="Delta in [V]",
    xaxis_title="Vin in [V]",
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)



fig.show()


